## Standard Continuous Time Lyapunov Equation

In [66]:
import scipy
import control
import numpy as np
import sympy as sp
from sympy.physics.vector import dynamicsymbols as dynamicsymbols

from pydrake.all import (LinearQuadraticRegulator, MathematicalProgram, Variables,
                         Solve, RealContinuousLyapunovEquation)

In [5]:
f = lambda x: [- x[1], x[0] + (x[0]**2 - 1) * x[1]]

In [6]:
A = np.array([[0, -1], [1, -1]])
Q = np.eye(2)

In [9]:
X1 = control.lyap(A.T, Q)

In [10]:
X1

array([[ 1.5, -0.5],
       [-0.5,  1. ]])

In [21]:
E = np.eye(2)
E[0, 1] = 2

In [22]:
X2 = control.lyap(A.T, Q, E = E)

In [23]:
X2

array([[-3.5,  1.5],
       [ 1.5, -1. ]])

## Cart Pole Dynamics

For reference of dynamics, see Shen Shen's quotient ring optimization paper. 
Assume unit mass and unit length

Write the dynamics as 
M $\times$ y = F

In [31]:
g, t = sp.symbols('g t')
u = sp.symbols('u')
q0, q1 = dynamicsymbols('q0 q1')

In [63]:
q0_dot = sp.diff(q0, t)
q1_dot = sp.diff(q1, t)

q0_dotdot = sp.diff(q0_dot, t)
q1_dotdot = sp.diff(q1_dot, t)

In [64]:
# state vector
y = sp.Matrix([q0, q1, q0_dot, q1_dot])
y_dot = sp.Matrix([q0_dot, q1_dot, q0_dotdot, q1_dotdot])

# force vector
F = sp.Matrix([q0_dot, q1_dot, q1_dot*sp.cos(q1), -1*g*sp.cos(q1)])

# mass matrix
M = sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 2, -sp.sin(q1)], [0, 0, -sp.sin(q1), 1]]) 

## Cart Pole Explicitly

Write the dynamics as $\dot{y}$ = f(y)

In [68]:
f = sp.simplify(M.inv()*F)

In [78]:
# linearize now
A = f.jacobian(y)

# substitute values at equilibrium point
sp.simplify(A.subs([(q1, sp.pi/2), (g, 10)]))

Matrix([
[0,  0, 1, 0],
[0,  0, 0, 1],
[0, 10, 0, 0],
[0, 20, 0, 0]])

In [79]:
A = np.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, 10, 0, 0], [0, 20, 0, 0]])
Q = np.eye(4)

In [80]:
X1 = control.lyap(A.T, Q)

SlycotResultWarning: 
The matrices `A` and `-A'` have common or very close eigenvalues

In [81]:
P = RealContinuousLyapunovEquation(A, Q)

RuntimeError: RealContinuousLyapunovEquation(): Solution is not unique!

## Cart Pole Implicitly

In [85]:
A = F.jacobian(y)
A.subs([(q1, sp.pi/2)])

Matrix([
[0,                    0, 1, 0],
[0,                    0, 0, 1],
[0, -Derivative(pi/2, t), 0, 0],
[0,                    g, 0, 0]])

In [83]:
E = np.eye(4)
E[2, 2] = 2

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 1.]])

In [86]:
A = np.array([[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0], [0, 10, 0, 0]])

In [88]:
X2 = control.lyap(A.T, Q, E = E)

SlycotResultWarning: 
DICO = 'C' and the pencil A - lambda * E has a
degenerate pair of eigenvalues. That is, lambda_i =
-lambda_j for some i and j, where lambda_i and
lambda_j are eigenvalues of A - lambda * E. Hence,
equation (1) is singular;  perturbed values were
used to solve the equation (but the matrices A and
E are unchanged).

In [89]:
import math

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (AddMultibodyPlantSceneGraph, ControllabilityMatrix,
                         DiagramBuilder, Linearize, LinearQuadraticRegulator,
                         MeshcatVisualizerCpp, Parser, Saturation, SceneGraph,
                         Simulator, StartMeshcat, WrapToSystem)
from pydrake.examples.acrobot import (AcrobotGeometry, AcrobotInput,
                                      AcrobotPlant, AcrobotState)
from pydrake.examples.quadrotor import (QuadrotorGeometry, QuadrotorPlant,
                                        StabilizingLQRController)
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve

from underactuated import FindResource, running_as_notebook
from underactuated.meshcat_cpp_utils import MeshcatSliders
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

if running_as_notebook:
    mpld3.enable_notebook()


In [90]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [91]:
def cartpole_balancing_example():
    def UprightState():
        state = (0, np.pi, 0, 0)
        return state

    def Controllability(plant):
        context = plant.CreateDefaultContext()
        plant.get_actuation_input_port().FixValue(context, [0])
        plant.SetPositionsAndVelocities(context, UprightState())

        linearized_plant = Linearize(
            plant,
            context,
            input_port_index=plant.get_actuation_input_port().get_index(), output_port_index=plant.get_state_output_port().get_index())
        print(linearized_plant.A())
        print(linearized_plant.B())
        print(
            f"The singular values of the controllability matrix are: {np.linalg.svd(ControllabilityMatrix(linearized_plant), compute_uv=False)}"
        )

    def BalancingLQR(plant):
        # Design an LQR controller for stabilizing the CartPole around the upright.
        # Returns a (static) AffineSystem that implements the controller (in
        # the original CartPole coordinates).

        context = plant.CreateDefaultContext()
        plant.get_actuation_input_port().FixValue(context, [0])

        plant.SetPositionsAndVelocities(context, UprightState())

        Q = np.diag((10., 10., 1., 1.))
        R = np.array([1])

        # MultibodyPlant has many (optional) input ports, so we must pass the
        # input_port_index to LQR.
        return LinearQuadraticRegulator(
            plant,
            context,
            Q,
            R,
            input_port_index=plant.get_actuation_input_port().get_index())


    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.1)
    file_name = FindResource("models/cartpole.urdf")
    Parser(plant).AddModelFromFile(file_name)
    plant.Finalize()

    controller = builder.AddSystem(BalancingLQR(plant))
    builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0),
                    plant.get_actuation_input_port())

    # Setup visualization
    meshcat.Delete()
    meshcat.Set2dRenderMode(xmin=-2.5, xmax=2.5, ymin=-1.0, ymax=2.5)
    MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyMutableContextFromRoot(context)

    # Simulate
    simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
    duration = 5.0 if running_as_notebook else 0.1
    for i in range(5):
        context.SetTime(0.)
        plant.SetPositionsAndVelocities(
            plant_context,
            UprightState() + 0.1 * np.random.randn(4,))
        simulator.Initialize()
        simulator.AdvanceTo(duration)

np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
cartpole_balancing_example()

In [94]:
def UprightState():
    state = (0, np.pi, 0, 0)
    return state


builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.1)
file_name = FindResource("models/cartpole.urdf")
Parser(plant).AddModelFromFile(file_name)
plant.Finalize()


context = plant.CreateDefaultContext()
plant.get_actuation_input_port().FixValue(context, [0])
plant.SetPositionsAndVelocities(context, UprightState())

linearized_plant = Linearize(
    plant,
    context,
    input_port_index=plant.get_actuation_input_port().get_index(), output_port_index=plant.get_state_output_port().get_index())

print(linearized_plant.A())
print(linearized_plant.B())
print(
    f"The singular values of the controllability matrix are: {np.linalg.svd(ControllabilityMatrix(linearized_plant), compute_uv=False)}"
)

[[1.0000 0.0098 0.1000 0.0000]
 [0.0000 1.2158 0.0000 0.1000]
 [0.0000 0.0981 1.0000 0.0000]
 [0.0000 2.1582 0.0000 1.0000]]
[[0.0010]
 [0.0020]
 [0.0100]
 [0.0200]]
The singular values of the controllability matrix are: [0.0735 0.0069 0.0009 0.0000]
